In [ ]:
import os
import ot
import sys
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import kstest
module_path = os.path.abspath(os.path.join('..','scripts'))
if module_path not in sys.path:
    sys.path.append(module_path)
from W2ChangePoints import ChangePointDetector
data_path = os.path.abspath(os.path.join('..', 'data', 'Langevin_1D.txt'))
truth_path = os.path.abspath(os.path.join('..', 'data', 'ChangePts.txt'))

In [ ]:
data = np.loadtxt(data_path)`` bi
truth_data = np.loadtxt(truth_path)

In [ ]:
window_size = 1000
ks_data = [kstest(data[(t - window_size):t], data[t:(t + window_size)]) for t in range(window_size, len(data) - window_size)]
ks_stats = [x.statistic for x in ks_data]
ks_pvals = [x.pvalue for x in ks_data]
dw_data = [ot.emd2_1d(data[(t - window_size):t], data[t:(t + window_size)]) for t in range(window_size, len(data) - window_size)]
change_points = [t in truth_data for t in range(len(data))]

In [ ]:
ks_tp, ks_fp, ks_fn, dw_tp, dw_fp, dw_fn = 0,0,0,0,0,0
ks_cutoff = 0.9
dw_cutoff = 0.85
tolerance = 100
ks_cps = [x > ks_cutoff for x in ks_stats]
dw_cps = [x > dw_cutoff for x in dw_data]
for t in range(len(data) - 2 * window_size):
    # if we detected a change point and there IS a true change point in the tolerance window, tp++
    if ks_cps[t] and True in change_points[(t + window_size - tolerance):(t + window_size + tolerance)]:
        ks_tp +=1
    if dw_cps[t] and True in change_points[(t + window_size - tolerance):(t + window_size + tolerance)]:
        dw_tp +=1
    # if we detected a change point and there IS NOT a true change point in the tolerance window, fp++
    if ks_cps[t] and True not in change_points[(t + window_size - tolerance):(t + window_size + tolerance)]:
        ks_fp +=1
    if dw_cps[t] and True not in change_points[(t + window_size - tolerance):(t + window_size + tolerance)]:
        dw_fp +=1
    # if we did not detect and change point and there IS a true change point int he tolerance window, fn++
    if not ks_cps[t] and True in change_points[(t + window_size - tolerance):(t + window_size + tolerance)]:
        ks_fn +=1
    if not dw_cps[t] and True in change_points[(t + window_size - tolerance):(t + window_size + tolerance)]:
        dw_fn +=1
ks_recall = ks_tp / (ks_tp + ks_fn)
dw_recall = dw_tp / (dw_tp + dw_fn)
ks_precision = ks_tp / (ks_tp + ks_fp)
dw_precision = dw_tp / (dw_tp + dw_fp)
ks_f1 = 2 / ((1 / ks_recall) + (1 / ks_precision))
dw_f1 = 2 / ((1 / dw_recall) + (1 / dw_precision))
print(ks_recall, ks_precision, ks_f1)
print(dw_recall, dw_precision, dw_f1)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(15,10)
#ax.plot(np.arange(window_size, len(data) - window_size), ks_stats, c='b')
for t in range(window_size, len(data) - window_size):
    if ks_stats[t - window_size] > 0.9:
        ax.vlines(t, -1.5, 1.5, colors='blue',linestyle='dashdot')
ax.plot(np.arange(len(data)), data, c='orange')
plt.title("Change Points Detected via Kolmogorov-Smirnov Statistics")

In [ ]:
plt.clf()
fig, ax = plt.subplots()
fig.set_size_inches(15,10)
cutoff = np.quantile(dw_data, 0.85)
#ax.plot(np.arange(window_size, len(data) - window_size), ks_stats, c='b')
for t in range(window_size, len(data) - window_size):
    if dw_data[t - window_size] > cutoff:
        ax.vlines(t, -1.5, 1.5, colors='blue',linestyle='dashdot')
ax.plot(np.arange(len(data)), data, c='orange')
plt.title("Change Points Detected via Wasserstein Differentials")